# Environment

In [1]:
import sys
from os import path
sys.path.append(path.dirname(path.dirname(path.abspath("Test.ipynb"))))
import time
import logging
from Controllers_Definition import testcontroller1, testcontroller2, testcontroller3, testcontroller4
from datetime import datetime
from FMLC.triggering import triggering
from FMLC.baseclasses import eFMU
from FMLC.stackedclasses import controller_stack
import random

In [2]:
logger = logging.getLogger(__name__)
'''
import matplotlib.pyplot as plt
%matplotlib inline
'''
logger.setLevel(logging.DEBUG)

# Triggering Class

In [4]:
ts = {} 
ts['main'] = 0.5 # seconds
ts['print'] = 1 # seconds

trigger_test = triggering(ts)
now_init = time.time()
now = now_init
while now < now_init+2:
    now = time.time()
    if now >= trigger_test.trigger['main']:
        print ('Main triggered', now)
        trigger_test.refresh_trigger('main', now)
    if now >= trigger_test.trigger['print']:
        print ('Print triggered', now)
        trigger_test.refresh_trigger('print', now)

Main triggered 1613366397.0
Main triggered 1613366397.5
Main triggered 1613366398.0
Print triggered 1613366398.0
Main triggered 1613366398.5


# Controller Base Class (eFMU)

In [5]:
# Test controller
testcontroller = testcontroller1() #this controller multiplies the inputs
# Get all variables
variables = testcontroller.get_model_variables()
# Makeup some inputs
inputs = {}
for var in variables:
    inputs[var] = random.randint(1,50)
# Query controller
print ('Log-message', testcontroller.do_step(inputs=inputs))
print ('Input', testcontroller.input)
print ('Output', testcontroller.output)
print('Output', testcontroller.get_var('output'))

Log-message testcontroller1 did a computation!
Input {'a': 23, 'b': 30}
Output {'c': 690}
Output {'c': 690}


# Controller Stack Class (single-thread/multi-thread)

In [7]:
# create a mapping of controllers and their sample times
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}

# Create a mapping of inputs for each controller
mapping = {}
mapping['forecast1_a'] = 10
mapping['forecast1_b'] = 4
mapping['forecast2_a'] = 20
mapping['forecast2_b'] = 4
mapping['forecast3_a'] = 30
mapping['forecast3_b'] = 4
mapping['mpc1_a'] = 'forecast1_c'
mapping['mpc1_b'] = 'forecast1_a'
mapping['control1_a'] = 'mpc1_c'
mapping['control1_b'] = 'mpc1_a'

## Single Thread

In [7]:
# Initialize the controller_stack using the mappings above
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=False, timeout=2)

# Call query_control 6 times. We should expect 6 records(excluding NaN) for each controller.
# In single thread mod, each call of query_control will trigger a computations for each controller in the system.
for i in range(6):
    ctrl_stack.query_control(time.time())
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-14 21:19:52.055431936,NaN,NaN,NaN,Initialize
2021-02-14 21:20:11.905138944,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:20:13.182643968,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:20:14.466065920,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:20:15.742275072,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:20:17.021395968,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:20:18.298760960,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-14 21:19:52.055431936,NaN,NaN,NaN,Initialize
2021-02-14 21:20:11.905138944,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:20:13.182643968,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:20:14.466065920,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:20:15.742275072,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:20:17.021395968,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:20:18.298760960,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-14 21:19:52.055431936,NaN,NaN,NaN,Initialize
2021-02-14 21:20:11.905138944,400.0,40.0,16000.0,testcontroller1 did a computation!
2021-02-14 21:20:13.182643968,400.0,40.0,16000.0,testcontroller1 did a computation!
2021-02-14 21:20:14.466065920,400.0,40.0,16000.0,testcontroller1 did a computation!
2021-02-14 21:20:15.742275072,400.0,40.0,16000.0,testcontroller1 did a computation!
2021-02-14 21:20:17.021395968,400.0,40.0,16000.0,testcontroller1 did a computation!
2021-02-14 21:20:18.298760960,400.0,40.0,16000.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-14 21:19:52.055431936,NaN,NaN,NaN,Initialize
2021-02-14 21:20:11.905138944,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:20:13.182643968,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:20:14.466065920,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:20:15.742275072,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:20:17.021395968,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:20:18.298760960,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-14 21:19:52.055431936,NaN,NaN,NaN,Initialize
2021-02-14 21:20:11.905138944,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:20:13.182643968,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:20:14.466065920,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:20:15.742275072,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:20:17.021395968,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:20:18.298760960,30.0,4.0,120.0,testcontroller1 did a computation!


## Multi Thread

In [8]:
# Initialize another controller_stack using the mappings above. This time, parallel = True
controllers = {}
controllers['forecast1'] = {'fun':testcontroller1, 'sampletime':1}
controllers['mpc1'] = {'fun':testcontroller2, 'sampletime':'forecast1'}
controllers['control1'] = {'fun':testcontroller1, 'sampletime':'mpc1'}
controllers['forecast2'] = {'fun':testcontroller3, 'sampletime':1}
controllers['forecast3'] = {'fun':testcontroller1, 'sampletime':1}
ctrl_stack = controller_stack(controllers, mapping, tz=-8, debug=True, parallel=True, timeout=2)


# Call query_control 6 times. We should expect there are 6 records(excluding NaN) for each task.
# In multi thread mod, each call of query_control will trigger a computation for one controller within each task. 
# We assign tasks based on input dependency. Since the inputs of mpc1 and control1 depend on the out put of
# forecast1, they are in the same task as forecast1.
for i in range(6):
    ctrl_stack.query_control(time.time())
    time.sleep(1.5)
for df in ctrl_stack.log_to_df().values():
    display(df)

,a,b,c,Logging
2021-02-14 21:21:31.317922048,NaN,NaN,NaN,Initialize
2021-02-14 21:22:07.455787008,10.0,4.0,40.0,testcontroller1 did a computation!
2021-02-14 21:22:12.106528000,10.0,4.0,40.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-14 21:21:31.317922048,NaN,NaN,NaN,Initialize
2021-02-14 21:22:08.999105024,40.0,10.0,400.0,testcontroller2 did a computation!
2021-02-14 21:22:13.656392960,40.0,10.0,400.0,testcontroller2 did a computation!


,a,b,c,Logging
2021-02-14 21:21:31.317922048,NaN,NaN,NaN,Initialize
2021-02-14 21:22:10.552188928,400.0,40.0,16000.0,testcontroller1 did a computation!


,a,b,c,Logging
2021-02-14 21:21:31.317922048,NaN,NaN,NaN,Initialize
2021-02-14 21:22:07.455787008,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:22:10.552188928,20.0,4.0,80.0,testcontroller3 did a computation!
2021-02-14 21:22:13.656392960,20.0,4.0,80.0,testcontroller3 did a computation!


,a,b,c,Logging
2021-02-14 21:21:31.317922048,NaN,NaN,NaN,Initialize
2021-02-14 21:22:07.455787008,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:22:08.999105024,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:22:10.552188928,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:22:12.106528000,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:22:13.656392960,30.0,4.0,120.0,testcontroller1 did a computation!
2021-02-14 21:22:15.207249920,30.0,4.0,120.0,testcontroller1 did a computation!
